In [14]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [15]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c, Habilitado__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
        "Habilitado__c",
    ]
]
Empresas_SF.shape

(7008, 11)

In [16]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6715, 5)

In [17]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(7014, 16)

In [18]:
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Empleador__c": "Empleador",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",        
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
        "Vigencia__c": "Vigente",
        "Corredor__c": "Corredor",
        "Pa_s_del_Paciente__c": "Pais",
        "Habilitado__c": "Habilitado",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Empleador",
        "Holding",
        "Campaña",        
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
        "Habilitado",
    ]
]

In [19]:

Programas_empresas.to_excel("C:/Users/jmartinez/Documents/Empresas2.xlsx", index=False)
#.to_excel("F:/Empresas SF/reportes/empresas2.xlsx", index=False)
Programas_empresas.head()

,Id Account Empresa,Nombre Empresa,Empleador,Holding,Campaña,Poliza,Programas_por_Empresas__c,Codigo EMP,Codigo A,Vigente,Corredor,Pais,Habilitado
0,0018c00002f27paAAA,Forus - Dado De Baja - Sin Servicio,Forus - Dado De Baja - Sin Servicio,Sin Holding Asociado,Forus,Sin Póliza Asociada,a1W8c0000083oJOEAY,EMP9004,A-8034,NO,None,a1U8c000005dAtnEAE,False
1,0018c00002ghxCCAAY,BICE Vida S.A.,BICE Vida S.A.,Sin Holding Asociado,BICE Vida,Sin Póliza Asociada,a1W8c0000083qXGEAY,EMP9090,A-8117,None,None,a1U8c000005dAtnEAE,True
2,0018c00002cY37kAAC,Mercado Libre Ltda.,Mercado Libre Ltda.,Mercado Libre,Sura,7720,a1W8c0000083jGVEAY,EMP5617,A-2356,SI,None,a1U8c000005dAtnEAE,True
3,0018c00002f2W7UAAU,Minera Escondida Ltda.,Minera Escondida Ltda.,Grupo BHP,Vida Security,284247,a1W8c0000083rZnEAI,EMP9011,A-9239,SI,None,a1U8c000005dAtnEAE,True
4,0018c00002f2WJyAAM,Minera Escondida Ltda.,Minera Escondida Ltda.,Grupo BHP,Vida Security,282350,a1W8c0000083rZoEAI,EMP9013,A-9240,SI,None,a1U8c000005dAtnEAE,True
